In [10]:
import collections
import functools
import math
import operator
import pandas as pd
import random

### Get a sieve of all numbers $n$ to indicate whether they are prime, with $0 \leq n \leq 10^5$

In [6]:
def get_primes_eratosthenes_sieve(n):
    is_prime = [1] * (n + 1)
    p = 2
    is_prime[0] = 0
    is_prime[1] = 0
    while p * p <= n:
        if is_prime[p]:
            for i in range(p * 2, n + 1, p):
                is_prime[i] = 0
        p += 1
    prime_list = []
    for i in range(n + 1):
        if is_prime[i]:
            prime_list.append(i)
    return is_prime

is_prime = get_primes_eratosthenes_sieve(10 ** 5)

### Use Pollard's Rho algorithm for finding prime factorizations for any $n$

In [7]:
# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if is_prime[x]:
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return {x: 1}
    factors = []
    pollard_rho_rec(x, factors)
    return collections.Counter(factors)

### Store prime factorizations for each $n$, with $2\leq n\leq10^5$ for rapid access 

In [24]:
N = 10 ** 5
prime_factorization = {n: pollard_rho_prime_factorization(n) for n in range(1, N + 1)}

### Maintain a table of $n$ and $\text{rad}(n)$ for $1\leq n\leq 10^5$ and sort as required

In [25]:
radicals_df = pd.DataFrame({'n': range(1, N + 1)})
radicals_df['rad'] = radicals_df.n.apply(lambda n: functools.reduce(operator.mul, [p for p in prime_factorization[n]]))
radicals_df.sort_values(['rad', 'n'], inplace=True)

In [26]:
def E(k):
    return radicals_df.iloc[k - 1].n

In [27]:
E(10000)

21417